# X-Ray Image Classification Demo

## 1. Download data and preparation

1.1 Clone the image files from Github

In [0]:
! git clone https://github.com/jsphchan/ChestX-ray8.git

1.2 List the X-Ray image files that belong to the class "No Finding".

In [0]:
! ls ChestX-ray8/images/test/0_nofinding

1.3 List the X-Ray image files that belong to the class "Effusion".

In [0]:
! ls ChestX-ray8/images/test/1_effusion

## 2. Train model using VGG16 model and test the accuracy



In [0]:
import numpy as np
from tensorflow import keras

hyperparameter_epoch = 5
hyperparameter_minibatch = 16
hyperparameter_lr = 0.0001

train_datagen = keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip=True,
        width_shift_range=0.15,
        height_shift_range=0.15,
        rotation_range=5,
        shear_range=0.01,
        zoom_range=0.10)
train_generator = train_datagen.flow_from_directory(
        'ChestX-ray8/images/train',
        batch_size=hyperparameter_minibatch,
        target_size=(224, 224),
        class_mode='binary')

val_datagen = keras.preprocessing.image.ImageDataGenerator()
val_generator = val_datagen.flow_from_directory(
        r'ChestX-ray8/images/val',
        batch_size=hyperparameter_minibatch,
        target_size=(224, 224),
        class_mode='binary',
        shuffle=False)

test_datagen = keras.preprocessing.image.ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
        'ChestX-ray8/images/test',
        batch_size=hyperparameter_minibatch,
        target_size=(224, 224),
        class_mode='binary',
        shuffle=False)

model_vgg16 = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3))

for layer in model_vgg16.layers[0:len(model_vgg16.layers)*9//10]:  
    layer.trainable = False

y = model_vgg16.output
y = keras.layers.GlobalAveragePooling2D()(y)
y = keras.layers.Dense(1, activation='sigmoid')(y)
model = keras.models.Model(model_vgg16.input, y)
model.summary()

adam = keras.optimizers.Adam(lr=hyperparameter_lr)
model.compile(
        optimizer=adam,
        loss=keras.losses.binary_crossentropy,
        metrics=['accuracy'])

train_step = train_generator.samples // hyperparameter_minibatch
val_step = val_generator.samples // hyperparameter_minibatch
test_step = test_generator.samples // hyperparameter_minibatch

model.fit_generator(generator=train_generator,
        steps_per_epoch=train_step,
        epochs=hyperparameter_epoch,
        validation_data=val_generator,
        validation_steps=val_step,
        verbose=1)

y_prob = model.predict_generator(generator=test_generator,
        steps=test_step)
y_pred = np.where(y_prob >= 0.5, 1, 0)
y_true = test_generator.classes.reshape(y_pred.shape)

print('Test Accuracy = %.4f' % np.mean(y_pred == y_true))

## 3. Try using the trained model to classify X-Ray image files

3.1 Test a sample X-Ray image file from "No Findings" class

In [0]:
import cv2
from google.colab.patches import cv2_imshow
image0 = cv2.imread('ChestX-ray8/images/test/0_nofinding/00000539_000.png')
image0_resize = cv2.resize(image0, (224, 224))
image0_batch = image0_resize[np.newaxis, ...]
print('Probability of Effusion   = %.4f' % np.squeeze(model.predict(image0_batch)))
print('Probability of No Finding = %.4f' % (1 - np.squeeze(model.predict(image0_batch))))
cv2_imshow(image0_resize)

3.2 Test a sample X-Ray image file from "Effusion" class

In [0]:
image1 = cv2.imread('ChestX-ray8/images/test/1_effusion/00022670_000.png')
image1_resize = cv2.resize(image1, (224, 224))
image1_batch = image1_resize[np.newaxis, ...]
print('Probability of Effusion   = %.4f' % np.squeeze(model.predict(image1_batch)))
print('Probability of No Finding = %.4f' % (1 - np.squeeze(model.predict(image1_batch))))
cv2_imshow(image1_resize)